In [1]:
import sqlite3 
from sqlite3 import Error 
import pandas as pd

In [2]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [11]:
connection = create_connection("SQL_booking_s_4.db")

Connection to SQLite DB successful


In [12]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

## Подготовка дашборда

In [52]:
!pip install dash

In [53]:
!pip install jupyter-dash

In [54]:
!pip install pandas

### Просмотр названий таблиц из базы данных

In [67]:
table = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'", connection)
print(table)

         name
0      Orders
1      Review
2       Users
3  Apart_info


### Просмотр полей таблиц из базы данных

In [69]:
c = connection.cursor()

def table_info(c, conn):
    '''
    prints out all of the columns of every table in db
    c : cursor object
    conn : database connection object
    '''
    tables = c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    for table_name in tables:
        table_name = table_name[0] # tables is a list of single item tuples
        table = pd.read_sql_query("SELECT * from {} LIMIT 0".format(table_name), conn)
        print(table_name)
        for col in table.columns:
            print('\t' + col)
        print()

table_info(c, connection)

Orders
	Booking_ID
	no_of_adults
	no_of_children
	no_of_weekend_nights
	no_of_week_nights
	type_of_meal_plan
	required_car_parking_space
	room_type_reserved
	lead_time
	year
	month
	day
	market_segment_type
	repeated_guest
	no_of_previous_cancellations
	no_of_previous_bookings_not_canceled
	avg_price_per_room
	no_of_special_requests
	booking_status
	user_id
	date_arrival
	apart_id

Review
	index
	Uniq Id
	Source
	Hotel Id From Source
	Hotel Url From Source
	Hotel Name
	Review Url
	Review Publication Date
	Review Author Type
	Review Author Name
	Review Author Location
	Review Author Age
	Review Author Count
	Review Stay Details
	Review Title
	Review Text Positive
	Review Text Negative
	Review Rating

Users
	ID
	Nationality
	Age
	DaysSinceCreation
	NameHash
	DocIDHash
	AverageLeadTime
	LodgingRevenue
	OtherRevenue
	BookingsCanceled
	BookingsNoShowed
	BookingsCheckedIn
	PersonsNights
	RoomNights
	DaysSinceLastStay
	DaysSinceFirstStay
	DistributionChannel
	MarketSegment
	SRHighFloor
	SRLow

In [85]:
meta = c.execute("PRAGMA table_info('orders')")
for r in meta:
    print(r)

(0, 'Booking_ID', 'TEXT', 0, None, 0)
(1, 'no_of_adults', 'INTEGER', 0, None, 0)
(2, 'no_of_children', 'INTEGER', 0, None, 0)
(3, 'no_of_weekend_nights', 'INTEGER', 0, None, 0)
(4, 'no_of_week_nights', 'INTEGER', 0, None, 0)
(5, 'type_of_meal_plan', 'TEXT', 0, None, 0)
(6, 'required_car_parking_space', 'INTEGER', 0, None, 0)
(7, 'room_type_reserved', 'TEXT', 0, None, 0)
(8, 'lead_time', 'INTEGER', 0, None, 0)
(9, 'year', 'INTEGER', 0, None, 0)
(10, 'month', 'INTEGER', 0, None, 0)
(11, 'day', 'INTEGER', 0, None, 0)
(12, 'market_segment_type', 'TEXT', 0, None, 0)
(13, 'repeated_guest', 'INTEGER', 0, None, 0)
(14, 'no_of_previous_cancellations', 'INTEGER', 0, None, 0)
(15, 'no_of_previous_bookings_not_canceled', 'INTEGER', 0, None, 0)
(16, 'avg_price_per_room', 'REAL', 0, None, 0)
(17, 'no_of_special_requests', 'INTEGER', 0, None, 0)
(18, 'booking_status', 'TEXT', 0, None, 0)
(19, 'user_id', 'INTEGER', 0, None, 0)
(20, 'date_arrival', 'TIMESTAMP', 0, None, 0)
(21, 'apart_id', 'INTEGER', 0

In [86]:
data = pd.read_sql('SELECT count() FROM orders', connection)
data

,count()
0,36275


In [91]:
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash()

df1 = pd.read_sql('SELECT date(date_arrival) as date, count(Booking_ID) as booking_number FROM orders WHERE year = 2017 GROUP BY date(date_arrival)', connection)

df2 = pd.read_sql('SELECT date(date_arrival) as date, count(Booking_ID) as booking_number FROM orders WHERE year = 2018 GROUP BY date(date_arrival)', connection)

fig = px.box(df1, y="booking_number")

app.layout = html.Div(children=[
    html.H1(children='Бронирование жилья', style={'text-align': 'center'}),
    html.Div(children='Динамика бронирований по датам', style={'text-align': 'center'}),
    
    html.Div([
        html.Label(['Выберите год:'],style={'font-weight': 'bold'}),
        dcc.Dropdown(
            id='dropdown',
            options=[
                {'label': '2017 год', 'value': 'graph1'},
                {'label': '2018 год', 'value': 'graph2'},
                    ],
            value='graph1',
            style={"width": "60%"}),
        
    html.Div(dcc.Graph(id='graph')),        
        ]),
    
    html.Div(dcc.Graph(id='boxplot')),        
        ])
    



@app.callback(
    Output('graph', 'figure'),
    [Input(component_id='dropdown', component_property='value')]
)
def select_graph(value):
    if value == 'graph1':
        fig1 = px.bar(df1, x=df1['date'], y=df1['booking_number'])
        return fig1
    else:
        fig2 = px.bar(x=df2['date'], y=df2['booking_number'])
        return fig2
    
@app.callback(
    Output('boxplot', 'figure'),
    [Input(component_id='dropdown', component_property='value')]
)
def select_graph(value):
    if value == 'graph1':
        fig1 = px.box(df1, y='booking_number', title = '"Ящик с усами" для числа бронирований')
        return fig1
    else:
        fig2 = px.box(df2, y='booking_number', title = '"Ящик с усами" для числа бронирований')
        return fig2
    


app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


In [ ]:
connection.close()